### Importações

In [2]:
import pandas as pd
import numpy as np

### Constantes

In [3]:
ABA_ANALISE = "MEXER_NESSA"
DATA_PATH   = "/Users/gabrielribeirobizerril/Documents/GitHub/llm/editai_extractor_llm_based/tratamento/_data/output_capitais_pt2.xlsx"
NAME_NAN    = ["não previu", "indisponível", "none", "ok", "nan", "na"]

### Carregamento

In [4]:
df = pd.read_excel(io=DATA_PATH, sheet_name=ABA_ANALISE)

In [5]:
df.columns

Index(['uf', 'path_pdf', 'pdf', 'document', 'chunks_relevantes',
       'texto_completo', 'valor_total', 'cotas_negras', 'cotas_indigenas',
       'cotas_pcd', 'vagas_totais', 'observações', 'VALOR', 'PN', 'PI', 'PCD',
       'PN2', 'PI2', 'PCD2'],
      dtype='object')

In [6]:
df.shape

(144, 19)

### Validação de Valores

In [7]:
# Excluir última linha (no excel era uma linha de valores totais)
df = df.iloc[:-1]

In [8]:
# Corrige os valores que estão vazios
df["cotas_negras"]    = df["cotas_negras"].apply(
    lambda x: np.nan if str(x).strip().lower() in NAME_NAN else x
)

df["cotas_indigenas"] = df["cotas_indigenas"].apply(
    lambda x: np.nan if str(x).strip().lower() in NAME_NAN else x
)

df["cotas_pcd"]       = df["cotas_pcd"].apply(
    lambda x: np.nan if str(x).strip().lower() in NAME_NAN else x
)

df["cotas_negras"] = df["cotas_negras"].astype(str).str.replace(",", ".", regex=False)
df["cotas_indigenas"] = df["cotas_indigenas"].astype(str).str.replace(",", ".", regex=False)
df["cotas_pcd"] = df["cotas_pcd"].astype(str).str.replace(",", ".", regex=False)

In [9]:
# Passa coluna de cotas para decimais 
df["cotas_negras"]    = df["cotas_negras"].astype(float) / 100
df["cotas_indigenas"] = df["cotas_indigenas"].astype(float) / 100
df["cotas_pcd"]       = df["cotas_pcd"].astype(float) / 100

### Filtros

In [10]:
df = df[["uf", "pdf", "cotas_negras", "cotas_indigenas", "cotas_pcd", "vagas_totais", "valor_total"]]

### Instrumento de Fomento

In [11]:
df["instrumento_fomento"] = df["pdf"].apply(
    lambda x: x.split("_")[2].split(".")[0]
)

### Calculo de Valor e Vagas

In [12]:
for categoria in ["negras", "indigenas", "pcd"]:
    df[f"valor_{categoria}"] = df[f"cotas_{categoria}"] * df["valor_total"]
    df[f"vagas_{categoria}"] = df[f"cotas_{categoria}"] * df["vagas_totais"]

In [13]:
df.describe()

,cotas_negras,cotas_indigenas,cotas_pcd,vagas_totais,valor_total,valor_negras,vagas_negras,valor_indigenas,vagas_indigenas,valor_pcd,vagas_pcd
count,125.000000,119.000000,123.000000,132.000000,1.430000e+02,1.250000e+02,116.000000,1.190000e+02,113.000000,1.230000e+02,114.000000
mean,0.269280,0.112381,0.058257,47.696970,1.776856e+06,4.931718e+05,12.813534,2.077035e+05,5.429104,9.991963e+04,2.721768
std,0.068288,0.039696,0.021869,65.167423,3.183502e+06,8.090976e+05,16.618149,3.522846e+05,7.298662,1.681021e+05,3.372575
min,0.200000,0.083300,0.040000,1.000000,1.000000e+05,2.500000e+04,0.250000,1.000000e+04,0.100000,5.000000e+03,0.050000
25%,0.250000,0.100000,0.050000,13.750000,4.764340e+05,1.250000e+05,3.750000,5.000000e+04,1.800000,2.641250e+04,1.000000
50%,0.250000,0.100000,0.050000,29.000000,8.000000e+05,2.250000e+05,7.500000,9.000000e+04,3.000000,4.803921e+04,1.825000
75%,0.250000,0.100000,0.050000,52.000000,2.000000e+06,6.000000e+05,13.875000,2.110367e+05,6.000000,1.200000e+05,3.000000
max,0.500000,0.250000,0.200000,510.000000,2.500000e+07,6.075000e+06,127.500000,2.500000e+06,51.000000,1.250000e+06,25.500000


### Rank dos Editais

In [30]:
agrupado = df.groupby("uf").agg(
    qtd_pdfs=("pdf", "nunique"),
    valor_total=("valor_total", "sum")
).reset_index()

In [31]:
agrupado = agrupado.sort_values(by="qtd_pdfs", ascending=False)

In [32]:
agrupado["valor_per_edital"] = agrupado["valor_total"]/agrupado["qtd_pdfs"]

In [33]:
agrupado["valor_per_edital"] = agrupado["valor_per_edital"].apply(lambda x: f"{x:,.2f}")
agrupado["valor_total"]      = agrupado["valor_total"].apply(lambda x: f"{x:,.2f}")


In [34]:
agrupado

,uf,qtd_pdfs,valor_total,valor_per_edital
8,FORTALEZA,12,"9,051,094.08","754,257.84"
13,MANAUS,9,"14,686,857.14","1,631,873.02"
10,JOÃO PESSOA,9,"5,290,000.00","587,777.78"
23,SÃO PAULO,9,"48,985,200.00","5,442,800.00"
21,SALVADOR,9,"18,210,000.00","2,023,333.33"
12,MACEIÓ,8,"6,619,233.40","827,404.18"
19,RIO BRANCO,7,"4,088,000.00","584,000.00"
14,NATAL,7,"5,208,000.00","744,000.00"
20,RIO DE JANEIRO,6,"36,740,000.00","6,123,333.33"
25,VITÓRIA,6,"2,340,558.70","390,093.12"


### Matriz das Cotas

In [36]:
df.columns

Index(['uf', 'pdf', 'cotas_negras', 'cotas_indigenas', 'cotas_pcd',
       'vagas_totais', 'valor_total', 'instrumento_fomento', 'valor_negras',
       'vagas_negras', 'valor_indigenas', 'vagas_indigenas', 'valor_pcd',
       'vagas_pcd'],
      dtype='object')